In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
import os
os.environ["OPENAI_API_KEY"]=""

In [ ]:
loader=PyPDFLoader("")

In [ ]:
loader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [ ]:
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True}

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

In [ ]:
#!pip install faiss_cpu

In [ ]:
documents = loader.load()

In [ ]:
documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
db = FAISS.from_documents(texts, model_norm)

In [ ]:
db

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template1= """You are an assistant for question-answering tasks. You will be given a question and context.
Use the following context to answer the question.
Follow the below guidelines:
1.Understand the question and its meaning.
2.Understand the semantic meaning of the context and what is it trying to convey.
3.Based on the meanings of question and the context, if both the intents match, generate an answer to the question which is completely based on the context.
4.If you are not able to find answer from the context , dont make up the answer ,say -'I dont know the answer to out-of-context question'.
5.Give the detailed answer including all the important points from the context provided.
  Context: {context}
    Question: {question}
    Answer :"""

In [ ]:
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=prompt_template1)

In [ ]:
from langchain.llms import OpenAI

In [ ]:
#!pip install openai==0.27.1

In [ ]:
llm=OpenAI()

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,return_source_documents="True",chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [ ]:
result=qa("What is langchain?")
print(result["result"])